In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-126-g3f152e5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅


In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

In [ ]:
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 11.4 MB/s 
     |████████████████████████████████| 97 kB 5.0 MB/s 
     |████████████████████████████████| 180 kB 53.1 MB/s 
     |████████████████████████████████| 140 kB 50.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import os

path = '/content/drive/MyDrive/data/'
annotations_path = path + 'annotations/'
images_path = path + 'images/'

annotations_files = [something for something in os.listdir(annotations_path) if not os.path.isdir(annotations_path + something)]
images_files = [something for something in os.listdir(images_path) if not os.path.isdir(images_path + something)]
annotations_files[:5], images_files[:5]

(['maksssksksss1.xml',
  'maksssksksss104.xml',
  'maksssksksss106.xml',
  'maksssksksss103.xml',
  'maksssksksss101.xml'],
 ['maksssksksss0.png',
  'maksssksksss1.png',
  'maksssksksss13.png',
  'maksssksksss116.png',
  'maksssksksss137.png'])

In [ ]:
import re

annotations_files.sort(key = lambda e: int(re.sub('[^0-9]', '', e)))
images_files.sort(key = lambda e: int(re.sub('[^0-9]', '', e)))
annotations_files[:5], images_files[:5]

(['maksssksksss0.xml',
  'maksssksksss1.xml',
  'maksssksksss2.xml',
  'maksssksksss3.xml',
  'maksssksksss4.xml'],
 ['maksssksksss0.png',
  'maksssksksss1.png',
  'maksssksksss2.png',
  'maksssksksss3.png',
  'maksssksksss4.png'])

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 10160, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 10160 (delta 1), reused 3 (delta 1), pack-reused 10154
Receiving objects: 100% (10160/10160), 10.45 MiB | 25.24 MiB/s, done.
Resolving deltas: 100% (7041/7041), done.


In [ ]:
pwd = !pwd # ['/kaggle/working']
pwd = pwd[0] + '/'

dataset_path = pwd + 'datasets/facemask/'
if not os.path.isdir(dataset_path):
    !mkdir -p {dataset_path}

new_images_path = dataset_path + 'images/'
new_labels_path = dataset_path + 'labels/'

if not os.path.isdir(new_images_path):
    !cp -rf {images_path} {dataset_path}
if not os.path.isdir(new_labels_path):
    !mkdir -p {new_labels_path}
!ls {dataset_path}

images	labels


In [ ]:
def get_yolo_format(pic_width, pic_height, x_min, y_min, x_max, y_max):
    x_center = (x_max + x_min) / (2 * pic_width)
    y_center = (y_max + y_min) / (2 * pic_height)
    width = (x_max - x_min) / pic_width
    height = (y_max - y_min) / pic_height
    return x_center, y_center, width, height

In [ ]:
import xml.etree.ElementTree as ET

labels = ['with_mask', 'mask_weared_incorrect', 'without_mask']
infos = [] # used <5. Compare images>

for annotations_file in annotations_files:
    label_file_name = annotations_file.split('.')[0] + '.txt'
    with open(new_labels_path + label_file_name, 'w') as label_file:
        root = ET.parse(annotations_path + annotations_file)
        pic_width = int(root.find('size').findtext('width'))
        pic_height = int(root.find('size').findtext('height'))
        info = [pic_width, pic_height]
        for obj in root.findall('object'):
            box_info = []
            class_name = obj.findtext('name')
            x_min = int(obj.find('bndbox').findtext('xmin'))
            y_min = int(obj.find('bndbox').findtext('ymin'))
            x_max = int(obj.find('bndbox').findtext('xmax'))
            y_max = int(obj.find('bndbox').findtext('ymax'))
            info.append([labels.index(class_name), x_min, y_min, x_max, y_max])
            yolo_format = get_yolo_format(pic_width, pic_height, x_min, y_min, x_max, y_max)
            label_file.write(str(labels.index(class_name)) + ' ' + ' '.join(map(str, yolo_format)) + '\n')
        infos.append(info)
        label_file.flush()

In [ ]:
labels_files = [something for something in os.listdir(new_labels_path) if not os.path.isdir(new_labels_path + something)]
labels_files.sort(key = lambda e: int(re.sub('[^0-9]', '', e)))

In [ ]:
!cat {new_labels_path}{labels_files[0]}

2 0.18359375 0.337431693989071 0.05859375 0.10109289617486339
0 0.4013671875 0.3333333333333333 0.080078125 0.12021857923497267
2 0.6689453125 0.3155737704918033 0.068359375 0.13934426229508196


In [ ]:
from sklearn.model_selection import train_test_split

show_count = 5 # used <5. Compare images>
images_train, images_else, labels_train, labels_else = train_test_split(images_files, labels_files, test_size = 0.2)
images_val, images_test, labels_val, labels_test = train_test_split(images_else, labels_else, test_size = show_count / len(images_else))

len(images_train), len(images_val), len(images_test)

(682, 166, 5)

In [ ]:
sub_directories = ['train/', 'val/', 'test/']
for sub_directory in sub_directories:
    if not os.path.isdir(new_images_path + sub_directory):
        !mkdir {new_images_path}{sub_directory}
    if not os.path.isdir(new_labels_path + sub_directory):
        !mkdir {new_labels_path}{sub_directory}
        
# move from all data to train
!mv {new_images_path}{images_files[0].split('.')[0][:-1]}* {new_images_path}train
!mv {new_labels_path}{labels_files[0].split('.')[0][:-1]}* {new_labels_path}train

In [ ]:
def move_data(source_directory, source_files, target_directory):
    for source_file in source_files:
        !mv {source_directory}{source_file} {target_directory}{source_file}

In [ ]:
images_files_list = [images_val, images_test]
labels_files_list = [labels_val, labels_test]

for images_files, labels_files, sub_directory in zip(images_files_list, labels_files_list, sub_directories[1:]):
    move_data(new_images_path + sub_directories[0], images_files, new_images_path + sub_directory)
    move_data(new_labels_path + sub_directories[0], labels_files, new_labels_path + sub_directory)

In [ ]:
import yaml

yaml_file = pwd + 'config.yaml'

yaml_data = dict(
    path = new_images_path,
    train = (new_images_path + sub_directories[0])[:-1],
    val = (new_images_path + sub_directories[1])[:-1],
    nc = len(labels),
    names = labels
)

with open(yaml_file, 'w') as f:
    yaml.dump(yaml_data, f, explicit_start = True, default_flow_style = False)

In [ ]:
%%writefile {yaml_file}

path: /kaggle/working/datasets/facemask/images/
train: /kaggle/working/datasets/facemask/images/train
val: /kaggle/working/datasets/facemask/images/val
nc: 3
names: ['with_mask', 'mask_weared_incorrect', 'without_mask']

Overwriting /content/yolov5/config.yaml


In [ ]:
!cat config.yaml


path: /kaggle/working/datasets/facemask/images/
train: /kaggle/working/datasets/facemask/images/train
val: /kaggle/working/datasets/facemask/images/val
nc: 3
names: ['with_mask', 'mask_weared_incorrect', 'without_mask']

In [ ]:
!ls

config.yaml	 detect.py   LICENSE	       runs	       utils
CONTRIBUTING.md  Dockerfile  models	       setup.cfg       val.py
data		 export.py   README.md	       train.py        yolov5
datasets	 hubconf.py  requirements.txt  tutorial.ipynb  yolov5s.pt


In [ ]:
yolo_path = 'yolov5/'
if not os.path.isdir(yolo_path):
    !git clone https://github.com/ultralytics/yolov5.git
!pip3 install -qr {yolo_path}requirements.txt

In [ ]:
import torch

model_name = 'yolov5l'
image_size = 640
batch_size = 16
epochs = 10
device = '0' if torch.cuda.is_available() else 'cpu'
saved_model_name = 'best.pt'

# for test
confidence_threshold = 0.25 # Threshold of object inference
iou_threshold = 0.45 # Threshold of remove overlapping boxes

device

'0'

In [ ]:
!python3 {yolo_path}train.py --weights {model_name}.pt \
        --cfg {yolo_path}models/{model_name}.yaml --data {yaml_file} \
        --hyp {yolo_path}data/hyps/hyp.scratch.yaml --epochs {epochs} --batch-size {batch_size} \
        --img-size {image_size} --device {device}

In [ ]:
!python train.py --img 640 --batch 16 --epochs 10 --data coco128.yaml --weights yolov5s.pt --cache


wandb: Currently logged in as: teera (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-126-g3f152e5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp3/weights/best.pt --img 640 --conf 0.25 --iou 0.45 --source test


detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=test, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-126-g3f152e5 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/5 /content/yolov5/test/001.jpg: 640x640 5 with_masks, Done. (0.034s)
image 2/5 /content/yolov5/test/002.jpg: 640x640 4 without_masks, Done. (0.034s)
image 3/5 /content/yolov5/test/003.jpg: 640x640 8 with_masks, Done. (0.034s)
image 4/5 /content/yolov5/test/004.jpg: 640x640 10 with_masks, 2 without_masks, Done. (0.034s)
image 5/5 /content/yolov5/test/005.jpg: 640x640 2 with_masks, 1 wi